In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math
import dateparser
import datefinder

2023-06-15 12:42:46.337085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


importing Jupyter notebook from imagePreProcessor.ipynb
importing Jupyter notebook from TesseractModify.ipynb


In [2]:
imagesFolderPath = '/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv4/Dataset/SampleImages/'

In [3]:
def preProcessDataFrame(df, image):
    #drop null values
    df.dropna(inplace=True)
    
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"] 
    df["y"] = df["top"] + df["height"]
    
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]
    
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0
    
    df["isPast"] = 0
    df["isFuture"] = 0

In [85]:
def verifyDate(text):
    
    dates = list(datefinder.find_dates(text, strict=True))
    
    if len(dates)==0:
        return False
    else:
        for date in dates:
            year = date.year
            presentYear = date.today().year
            if abs(year-presentYear)<=2:
                print(dates)
                return True
            else:
                return False

In [76]:
def findDate(dateDF):
    dates = []
    
    for index in dateDF.index:
        text = dateDF["text"][index]
        if verifyDate(text):
            dates.append(text)
            Dates = list(datefinder.find_dates(text))
            dateToday = Dates[0].today()
            if Dates[0]<=dateToday:
                dateDF["isPast"][index]=1
            else:
                dateDF["isFuture"][index]=1
            
    
    return dates

In [77]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = text.lower()
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [78]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [79]:
def allocateNeighbours(df, neighbourFields):
    for index in df.index:
        x1 = df["x"][index]/df["PageWidth"][index]
        y1 = df["y"][index]/df["PageHeight"][index]
        block = df["block_num"][index]
        neighbours = {}
        for index2 in df.index:
            x2 = df["x"][index2]/df["PageWidth"][index2]
            y2 = df["y"][index2]/df["PageHeight"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if dist<0.2:
                neighbours[df["text"][index2].lower()]=dist
            if abs(y1-y2)<0.2 and abs(block2-block)<=1<=1:
                neighbours[df["text"][index2].lower()] = dist
        for n in neighbourFields:
            if n in neighbours:
                df[n][index]=neighbours[n]

In [80]:
def addNeighbours(df):
    neighbourFields = ["date", "dated", "invoice", "delivery", "order", "due", "payment", "tax", "bill", "receipt", "issue"]
    for col in neighbourFields:
        df[col]=0
    allocateNeighbours(df, neighbourFields)
    

In [81]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [82]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [83]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [84]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [59]:
#dfList = batchCreate(imagesFolderPath)

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_17785/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


[[0, 2], [3, 5], [6, 9], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [16, 17], [18, 20], [21, 24], [25, 25], [26, 27], [28, 36], [37, 37], [38, 42], [43, 48], [49, 56], [57, 59], [60, 60], [61, 61], [62, 62], [63, 65], [66, 66], [67, 68], [69, 71], [72, 75], [76, 78], [79, 81], [82, 84], [85, 89], [90, 96], [97, 97], [98, 98], [99, 101], [102, 102], [103, 105], [106, 109], [110, 113], [114, 114], [115, 116], [117, 120], [121, 123], [124, 127], [128, 131], [132, 132], [133, 133], [134, 134], [135, 135], [136, 136], [137, 137], [138, 138], [139, 139], [140, 140], [141, 142], [143, 143], [144, 145], [146, 155], [156, 156], [157, 158], [159, 159], [160, 160], [161, 161], [162, 162], [163, 164], [165, 165], [166, 166], [167, 169], [170, 172], [173, 173], [174, 178], [179, 179], [180, 180], [181, 181], [182, 183], [184, 185], [186, 186], [187, 195], [196, 196], [197, 197], [198, 198], [199, 202], [203, 205], [206, 210], [211, 211], [212, 212], [213, 213], [214, 215], [216, 21

/tmp/ipykernel_17785/614737289.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["isPast"][index]=1
/tmp/ipykernel_17785/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_17785/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documen

[[0, 0], [1, 3], [4, 4], [5, 5], [6, 7], [8, 9], [10, 13], [14, 15], [16, 16], [17, 18], [19, 19], [20, 23], [24, 29], [30, 36], [37, 47], [48, 49], [50, 52], [53, 54], [55, 55], [56, 58], [59, 59], [60, 60], [61, 61], [62, 62], [63, 63], [64, 64], [65, 67], [68, 70], [71, 73], [74, 76], [77, 77], [78, 79], [80, 80], [81, 81], [82, 82], [83, 83], [84, 86], [87, 87], [88, 89], [90, 90], [91, 93], [94, 94], [95, 96], [97, 97], [98, 98], [99, 99], [100, 100], [101, 101], [102, 102], [103, 103], [104, 104], [105, 109], [110, 117], [118, 119], [120, 121], [122, 125], [126, 126], [127, 127], [128, 129], [130, 134], [135, 136], [137, 137], [138, 138], [139, 139], [140, 140], [141, 141], [142, 143]]
[datetime.datetime(2022, 9, 10, 0, 0)]
[datetime.datetime(2022, 10, 16, 0, 0)]
[datetime.datetime(2022, 9, 10, 0, 0)]
[datetime.datetime(2022, 10, 16, 0, 0)]
[datetime.datetime(2022, 9, 10, 0, 0)]
[datetime.datetime(2022, 10, 16, 0, 0)]
[datetime.datetime(2022, 9, 10, 0, 0)]
[datetime.datetime(2022

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_17785/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_17785/614737289.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["isPast"][index]=1
/tmp/ipykernel_17785/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

...

/tmp/ipykernel_17785/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_17785/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_17785/614737289.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

[[0, 2], [3, 3], [4, 6], [7, 12], [13, 17], [18, 18], [19, 21], [22, 23], [24, 24], [25, 25], [26, 26], [27, 27], [28, 28], [29, 30], [31, 33], [34, 35], [36, 37], [38, 40], [41, 43], [44, 45], [46, 46], [47, 48], [49, 49], [50, 50], [51, 51], [52, 52], [53, 56], [57, 58], [59, 59], [60, 60], [61, 66], [67, 69], [70, 71], [72, 72], [73, 75], [76, 77], [78, 79], [80, 83], [84, 84], [85, 92], [93, 94], [95, 96], [97, 97], [98, 100], [101, 107], [108, 117], [118, 133], [134, 149], [150, 164], [165, 174], [175, 175], [176, 176], [177, 177], [178, 178], [179, 179], [180, 180], [181, 181]]
[datetime.datetime(2023, 3, 14, 0, 0)]
[datetime.datetime(2023, 3, 14, 0, 0)]
[datetime.datetime(2023, 3, 14, 0, 0)]
[datetime.datetime(2023, 3, 14, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_17785/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


[[0, 0], [1, 1], [2, 8], [9, 9], [10, 10], [11, 11], [12, 13], [14, 16], [17, 21], [22, 25], [26, 28], [29, 30], [31, 33], [34, 40], [41, 43], [44, 46], [47, 50], [51, 52], [53, 58], [59, 62], [63, 63], [64, 64], [65, 67], [68, 68], [69, 69], [70, 70], [71, 71], [72, 74], [75, 75], [76, 79], [80, 82], [83, 85], [86, 90], [91, 92], [93, 93], [94, 96], [97, 98], [99, 107], [108, 108], [109, 112], [113, 113], [114, 116], [117, 119], [120, 120], [121, 122], [123, 132], [133, 133], [134, 135], [136, 137], [138, 139], [140, 140], [141, 144], [145, 147], [148, 148], [149, 149], [150, 150], [151, 151], [152, 152], [153, 153], [154, 154], [155, 156], [157, 159], [160, 161], [162, 166], [167, 167], [168, 168], [169, 169], [170, 170], [171, 171], [172, 172], [173, 173], [174, 174], [175, 175], [176, 178], [179, 179], [180, 189], [190, 190], [191, 207], [208, 209], [210, 218], [219, 224], [225, 226], [227, 228], [229, 230], [231, 231], [232, 244], [245, 258], [259, 266], [267, 267], [268, 268], [2

/tmp/ipykernel_17785/614737289.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["isPast"][index]=1
/tmp/ipykernel_17785/1269700397.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=neighbours[n]


In [60]:
#len(dfList)

53

In [61]:
#resultDF = pd.concat(dfList)

In [62]:
#resultDF.shape

(155, 32)

In [63]:
#len(resultDF["imageName"].unique())

44

In [64]:
#resultDF

,block_num,line_num,left,top,width,height,conf,text,nextDist,x,...,delivery,order,due,payment,tax,bill,receipt,issue,imageName,output
117,7,2,300,1105,155,23,96.191299,02.04.2023,440,455,...,0.0,0.151753,0.000000,0.000000,0.528837,0.0,0.0,0.0,20.jpeg,0
123,7,3,1379,1146,155,23,95.732468,02.04.2023,-1279,1534,...,0.0,0.804302,0.000000,0.000000,0.131721,0.0,0.0,0.0,20.jpeg,0
180,20,2,873,1833,106,18,69.672714,"02/04/2023,",316,979,...,0.0,0.000000,0.000000,0.162403,0.421051,0.0,0.0,0.0,20.jpeg,0
204,26,1,873,1904,106,18,96.788101,"02/04/2023,",-236,979,...,0.0,0.000000,0.000000,0.132623,0.000000,0.0,0.0,0.0,20.jpeg,0
313,7,0,1168,1145,332,24,94.731491,Invoice Date : 02.04.2023,-1279,1500,...,0.0,0.783751,0.000000,0.000000,0.112152,0.0,0.0,0.0,20.jpeg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,8,0,1343,520,150,23,67.342278,14-Mar-23,-1462,1493,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,26.jpeg,0
28,5,2,721,397,126,18,72.247528,:24-DEC-22,-759,847,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,46.jpeg,0
130,14,2,1337,1086,115,18,88.408142,27-DEC-22,-1363,1452,...,0.0,0.283171,0.021537,0.042880,0.000000,0.0,0.0,0.0,46.jpeg,0
283,5,0,579,396,252,19,87.402677,Invoice Date :24-DEC-22,-759,831,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,46.jpeg,0


In [65]:
#resultDF.reset_index(inplace=True)

In [66]:
#resultDF.drop(columns=["index", "nextDist", "block_num", "line_num"], inplace=True)

In [67]:
#resultDF.head()

,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,delivery,order,due,payment,tax,bill,receipt,issue,imageName,output
0,300,1105,155,23,96.191299,02.04.2023,455,1128,2339,1654,...,0.0,0.151753,0.0,0.000000,0.528837,0.0,0.0,0.0,20.jpeg,0
1,1379,1146,155,23,95.732468,02.04.2023,1534,1169,2339,1654,...,0.0,0.804302,0.0,0.000000,0.131721,0.0,0.0,0.0,20.jpeg,0
2,873,1833,106,18,69.672714,"02/04/2023,",979,1851,2339,1654,...,0.0,0.000000,0.0,0.162403,0.421051,0.0,0.0,0.0,20.jpeg,0
3,873,1904,106,18,96.788101,"02/04/2023,",979,1922,2339,1654,...,0.0,0.000000,0.0,0.132623,0.000000,0.0,0.0,0.0,20.jpeg,0
4,1168,1145,332,24,94.731491,Invoice Date : 02.04.2023,1500,1169,2339,1654,...,0.0,0.783751,0.0,0.000000,0.112152,0.0,0.0,0.0,20.jpeg,0


In [68]:
#pwd

'/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv4/Dataset/SampleImages'

In [71]:
#os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset/')

In [72]:
#pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Tesseract-Modelv5/Dataset'

In [73]:
#resultDF.to_csv("tesseract.csv")